In [6]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import pandas as pd
import PyPDF2
import requests
from tqdm.contrib.telegram import tqdm

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLineHorizontal
from PyPDF2 import PdfReader

from urllib.parse import unquote
from transliterate import to_latin
import os
from math import *

In [140]:
tqdm.pandas()

In [147]:
TOKEN = os.getenv('TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

In [3]:
BASE_URL = "https://n.ziyouz.com"

MAIN_URL = "https://n.ziyouz.com/kutubxona"

In [69]:

categ_links = {}

async def main():
    async with aiohttp.ClientSession() as session:
        async with session.get(MAIN_URL) as resp:
            if resp.status == 200:
                text = await resp.text()

                soup = bs4.BeautifulSoup(text, 'lxml')
                quotes = soup.find('div', class_="pd-categories-view").find_all('div', class_="col-sm-4 col-md-4")
                
                for sub in tqdm(quotes, token=TOKEN, chat_id=CHAT_ID):
                    try:
                        title = sub.find("h3").text
                        categ_links[title] = []

                        links = sub.find("div", class_="ph-subcategories").find_all("div", class_="pd-subcategory")

                        for i in links:
                            link = unquote(i.find('a')['href'])
                            page = ceil(int(i.find('small').text[1:-1])/50)
                            
                            categ_links[title].append({"link":BASE_URL+link, "pages":int(page)})
                            
                    except Exception as e:
                        print(e, title, link)

            elif resp.status != 404:
                await asyncio.sleep(60) # 1 minute

await main()
   

  0%|          | 0/9 [00:00<?, ?it/s]

In [73]:
del categ_links['Ziyouz.com audiokutubxonasi (29/2580)']

del categ_links['Библиотека Ziyouz.com (17/597)']

del categ_links['Qaraqalpaq kitapxanası (9/251)']

KeyError: 'Ziyouz.com audiokutubxonasi (29/2580)'

In [79]:
pdf_links = []

async def main():
    async with aiohttp.ClientSession() as session:
        for category in tqdm(categ_links, token=TOKEN, chat_id=CHAT_ID):
            try:
                for link in tqdm(categ_links[category], token=TOKEN, chat_id=CHAT_ID):
                    try:
                        for i in range(link['pages']):
                            try:
                                async with session.get(link['link']+f"?start={i*50}") as resp:
                                    if resp.status == 200:
                                        text = await resp.text()

                                        soup = bs4.BeautifulSoup(text, 'lxml')
                                        quotes = soup.find('div', class_="pd-category").find_all('a', class_="btn btn-success")

                                        for i in quotes:
                                            url = unquote(i['href'])
                                            pdf_links.append(BASE_URL+url)
                                    elif resp.status != 404:
                                        await asyncio.sleep(60) # 1 minute
                            except Exception as e:
                                print(e, "3rd block",link)
                            
                    except Exception as e:
                        print(e, "2nd block",link)
                
            except Exception as e:
                print(e, "1st block",link)

await main()

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [86]:
with open('HelperFiles/ziyo_books.txt', 'w') as file:
    file.write('\n'.join(pdf_links))

In [102]:
len(pdf_links)

17511

# Downloading PDF books

In [110]:
async def main():
#     connector = aiohttp.TCPConnector(force_close=True)
    async with aiohttp.ClientSession() as session:
        for link in tqdm(undefined_links, token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(link) as resp:
                    if resp.status == 200:
                        text = await resp.read()
                        with open('Ziyo_books/'+unquote(link.split('/')[-1])+".pdf", 'wb') as file:
                            file.write(text)
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(e, link)
await main()

  0%|          | 0/17511 [00:00<?, ?it/s]

CancelledError: 

# Extracting Text


In [4]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
import io
from transliterate import to_latin

In [5]:
def extract_text_from_pdf(file_path):
    resource_manager = PDFResourceManager()
    output_string = io.StringIO()
    codec = 'utf-8'
    laparams = None
    device = TextConverter(resource_manager, output_string, codec=codec, laparams=laparams)
    with open(file_path, 'rb') as file:
        interpreter = PDFPageInterpreter(resource_manager, device)
        for page in PDFPage.get_pages(file):
            interpreter.process_page(page)
        text = output_string.getvalue()
    device.close()
    output_string.close()
    return text

replaces = {"ª" : "Ў", "¡" : "Ғ", "²" : "Ҳ", "¯" : "Қ", "^" : "қ", "\xad" : ""}
def replace_values(text):
    for i,j in replaces.items():
        text = text.replace(i, j)
    return text

In [1]:
import glob
import os

pdf_folder = "Ziyo_books/"

all_pdfs = glob.glob(pdf_folder+"*.pdf")

In [3]:
for pdf_file in all_pdfs:
    if pdf_file[-8:] == ".pdf.pdf":
        os.rename(pdf_file, pdf_file.replace(".pdf.pdf", ".pdf"))
#     llinks.append()

In [ ]:
results = []

for pdf in tqdm(all_pdfs[26:], token=TOKEN, chat_id=CHAT_ID):
    try:
        text = extract_text_from_pdf(pdf).replace("\x0c", "")
        
        text = text.strip()
        
#         text = to_latin(text)
        
#         text = replace_values(text)
#         latin_text = obj.transliterate(text, from_="cyr", to="lat")
        results.append({"title": pdf[:-4],
                    "content": text})
    except Exception as e:
        print(e)

  0%|          | 0/17372 [00:00<?, ?it/s]

The PDF <_io.BufferedReader name='Ziyo_books/22-tarixiy-kitoblar?download=2700:majid-hasaniy-turkiston-bosqini&start=250.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


No /Root object! - Is this really a PDF?
No /Root object! - Is this really a PDF?
No /Root object! - Is this really a PDF?


The PDF <_io.BufferedReader name='Ziyo_books/41-o-zbek-nasri?download=393:asqad-muxtor-amu-roman&start=150.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='Ziyo_books/25-adabiyotshunoslik?download=121:alibek-rustamiy-so-z-haqida-so-z.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='Ziyo_books/152-tibbiyot?download=9144:klinik-farmakologiya-yu-mamadov&start=50.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='Ziyo_books/43-yoshlik?download=3281:yoshlik-2003-1&start=150.pdf

In [ ]:
df1 = pd.DataFrame(results)

df1.to_pickle("Ziyo_books.pickle")

In [50]:

df = pd.read_pickle("Ziyo_books.pickle")

<ipython-input-53-8c4846062760>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("www.ziyouz.com kutubxonasi", "")


In [54]:
df

,title,content
0,Ziyo_books/43-yoshlik?download=5020:yoshlik-20...,"Ўзбекиcтон Ёзувчилар уюшмаси, ""Камолот"" ёшлар ..."
1,Ziyo_books/25-adabiyotshunoslik?download=121:a...,
2,Ziyo_books/229-turkmancha?download=13197:5-syn...,«SHARQ» NEŞIRÝAT-ÇAPHANA PAÝDARLAR KOMPANIÝASY...
3,Ziyo_books/41-o-zbek-nasri?download=4369:naim-...,
4,Ziyo_books/149-sanat?download=5730:o-zbek-xalq...,O‘ZBEK XALQ CHOLG‘ULARIUCHUN PYESALARMadani...
...,...,...
9595,,
9596,Ziyo_books/14-jahon-nasri?download=3950:aleksa...,
9597,Ziyo_books/7-o-zbek-zamonaviy-she-riyati?downl...,
9598,,


# FILTERING DF 

In [164]:
import ray
ray.init()

2023-07-26 08:52:19,266	WARNING services.py:1832 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67067904 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-07-26 08:52:22,800	INFO worker.py:1621 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.6.1


In [43]:
import re

def filter_text_with_regex(text):
    # Define the updated regular expression pattern to match Latin, Cyrillic characters, and punctuation marks (excluding spaces).
    pattern = r"""[A-Za-zА-Яа-я0-9 _.,!?;:"'/$]*"""
    
    # Use re.findall() to find all occurrences of the pattern in the text.
    filtered_text = ''.join(re.findall(pattern, text))
    
    return filtered_text

In [44]:

# Example usage:
text_to_filter = "Hello, Привет! How are you? Как дела?"
filtered_text = filter_text_with_regex(text_to_filter)
# print(filtered_text)

In [62]:
df2 = pd.read_pickle("Ziyo_books.pickle")

In [65]:
df2['content'] = df2['content'].apply(lambda x: to_latin(x))

In [66]:
df2['content'] = df2['content'].apply(lambda x: filter_text_with_regex(x))

In [68]:
df2['content'] = df2['content'].str.replace("www.ziyouz.com kutubxonasi", "")

<ipython-input-68-13e7d084c501>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['content'] = df2['content'].str.replace("www.ziyouz.com kutubxonasi", "")


In [69]:
df2.replace(r'^\s*$', float('nan'), regex=True, inplace=True)

In [70]:
# Step 2: Drop rows containing at least one NaN (i.e., empty rows)
df2.dropna(axis=0, how='any', inplace=True)

In [72]:
from gradio_client import Client

In [165]:
@ray.remote
def uz_detect(text):
    try:
        client = Client("https://azzizz17-api-2.hf.space/")
        result = client.predict(
            text,  # str in 'text' Textbox component
            api_name="/predict"
            )
        result = eval(result)

        if result[0]['label'] in ['uz', 'uz_kr']:
            return text
        return ""
    
    except Exception as e:
        return ""

In [160]:
df3 = df2.copy()

In [161]:
def splitting(text):
    text = text.split(" ")
    text = [item for item in text if item != ""]
    text = text[40:70]
    
    return " ".join(text)
#     return text

In [162]:
df3['content'] = df3['content'].progress_apply(lambda x: splitting(x))

  0%|          | 0/5584 [00:00<?, ?it/s]

In [169]:
arr = [uz_detect.remote(text) for text in df3.content]

(uz_detect pid=27646) Loaded as API: https://azzizz17-api-2.hf.space/ ✔


In [170]:
result = ray.get(arr)

(uz_detect pid=27640) Loaded as API: https://azzizz17-api-2.hf.space/ ✔
(uz_detect pid=27632) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 11x across cluster]
(uz_detect pid=27619) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 11x across cluster]
(uz_detect pid=27629) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 11x across cluster]
(uz_detect pid=27634) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 9x across cluster]
(uz_detect pid=27630) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 8x across cluster]
(uz_detect pid=27643) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 8x across cluster]
(uz_detect pid=27650) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 11x across cluster]
(uz_detect pid=27628) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 10x across cluster]
(uz_detect pid=27643) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 11x across cluster]
(uz_detect pi

Exception ignored in: <function tqdm.__del__ at 0x7fa0aa932700>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/tqdm/contrib/telegram.py", line 139, in close
    if self.disable:
AttributeError: 'tqdm_telegram' object has no attribute 'disable'
Exception ignored in: <function tqdm.__del__ at 0x7fa0aa932700>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/tqdm/contrib/telegram.py", line 139, in close
    if self.disable:
AttributeError: 'tqdm_telegram' object has no attribute 'disable'


(uz_detect pid=27637) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 9x across cluster]
(uz_detect pid=27629) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 12x across cluster]
(uz_detect pid=27632) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 10x across cluster]
(uz_detect pid=27619) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 10x across cluster]
(uz_detect pid=27640) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 10x across cluster]
(uz_detect pid=27626) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 10x across cluster]
(uz_detect pid=27642) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 11x across cluster]
(uz_detect pid=27647) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 10x across cluster]
(uz_detect pid=27626) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 11x across cluster]
(uz_detect pid=27645) Loaded as API: https://azzizz17-api-2.hf.space/ ✔ [repeated 1

In [171]:
result

['NAZAROrozboy ABDURAHMONOVSahifalovchi: Giyosiddin OMON ogliUshbu son Yoshlik jurnali tahririyatining kompyuter markazida sahifalandi.Manzilimiz:Toshkent. sh.Ozbekiston shoh kochasi, 16"a" uy.Email: yoshlikxatlarrambler.ru Tel/faks: 8371 2270227, 2455793, 2450552MULOQOT Usmon AZIM. Adabiyotni halol odamlar yaratadi. ..............................',
 'NPK Ba redaksiasy, 2015UOK: 372.8075KBK 74.263M47 Zhmet okuwy: Umumy orta bilim bern mekdepleri 5nji synpy in derslik / Awtorlar . aripow we bag. D.: Sharq, 2015. 240 s.ISBN 9789943009233 UOK:',
 'pyesalar: Madaniyat, sanat kollejlari hamda akad. litseylariuchun oquv qoll./ Tuzuvchimuallif F. Ibrohimov; Maxsus muharrir N. Sharipov; OzR oliy vaorta maxsus talim vazirligi, orta maxsus, kasbhunar talimi markazi. T.: Yangi nashr,',
 'Ikki jildlik Oliy matematika, I va Oliy matematika, II darsliklari oliy texnik oquv yurtlarining bakalavrlar tayyorlash uchun oliy matematika fani boyicha tasdiqlangan oquv dasturi asosida yozilgan. Darslikning b

In [172]:
df3['content'] = result

In [174]:
df3.replace(r'^\s*$', float('nan'), regex=True, inplace=True)

# Step 2: Drop rows containing at least one NaN (i.e., empty rows)

df3.dropna(axis=0, how='any', inplace=True)

In [175]:
df3

,title,content
0,Ziyo_books/43-yoshlik?download=5020:yoshlik-20...,NAZAROrozboy ABDURAHMONOVSahifalovchi: Giyosid...
2,Ziyo_books/229-turkmancha?download=13197:5-syn...,"NPK Ba redaksiasy, 2015UOK: 372.8075KBK 74.263..."
4,Ziyo_books/149-sanat?download=5730:o-zbek-xalq...,"pyesalar: Madaniyat, sanat kollejlari hamda ak..."
5,Ziyo_books/145-matematika?download=7340:oliy-m...,"Ikki jildlik Oliy matematika, I va Oliy matema..."
7,Ziyo_books/125-ijtimoiy-fanlar?download=4510:o...,"suщestvovaniya, V interesax dalneyshego uluchs..."
...,...,...
9585,Ziyo_books/43-yoshlik?download=7880:yoshlik-20...,NAZAROrozboy ABDURAHMONOVSahifalovchi: Giyosid...
9586,Ziyo_books/167-guliston?download=7754:guliston...,", . . . , , . , , . , . , , . , Guliston22009/..."
9589,Ziyo_books/23-prezident-asarlari?download=1115...,chatsiriq Ozbekiston Respublikasi Oliy Majlisi...
9590,Ziyo_books/210-matbuot-solnoma?download=9923:o...,Vыxodit yejemesyachno 2009 2 Toshkent 2 BBK 91...


In [185]:
df4 = df2[df2['title'].isin(df3['title'])]

In [187]:
df4.to_pickle('MainFiles/Ziyo_books.pickle')